In [1]:
# Importacao das libraries
import pandas as pd
import functions as main
import numpy as np
from scipy import stats
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [2]:
ciclo = 4

opencsv = pd.read_csv(rf"D:\OneDrive - Instituto Politecnico de Castelo Branco\002_Learning\001_Pos_Graduacao\Ciclo 5 - Projeto Final\ProjetoFinal-PosGraduacaoEngenhariaFinanceira\ciclo{ciclo}_df.csv", sep=",")
opencsv.set_index('Date', inplace=True)
opencsv.index = pd.to_datetime(opencsv.index)
opencsv = opencsv.asfreq(pd.infer_freq(opencsv.index))
opencsv = opencsv.astype(float)

In [4]:
df_model = opencsv.copy()
#df_model.drop(columns=["BTC_USD"])

In [5]:
df_model.isnull().sum()

BalanceSheet_global    0
BTC_USD                0
Mining_Difficulty      0
USD_OnChain_Volume     0
GDP_usa                0
                      ..
L2.BTC_PageViews       0
L1.Crypto_PageViews    0
L2.Crypto_PageViews    0
L1.BTC_Supply          0
L2.BTC_Supply          0
Length: 63, dtype: int64

### DOLS

In [4]:
from statsmodels.tsa.api import add_trend

def dols(y, x, trend='c', max_lag=1, max_lead=1):
    y = add_trend(y, trend=trend)
    x = add_trend(x, trend=trend)
    x_lags = sm.tsa.lagmat(x, max_lag, trim='both')
    x_leads = sm.tsa.lagmat(x[::-1], max_lead, trim='both')[::-1]
    x_dols = np.hstack([x, x_lags, x_leads])
    model = sm.OLS(y[max_lag:-max_lead], x_dols[max_lag:-max_lead])
    return model.fit()

result = dols(opencsv['BTC_USD'], df_model)
print(result.summary())


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 206 and the array at index 1 has size 205

## Matriz de Correlacoes


In [ ]:
test_corr = df_model.copy()
test_corr = test_corr.iloc[:, :18] # apenas as variaveis originais
test_corr.shape

In [ ]:
main.correlation_map(df=test_corr, threshold=0.3)

## Seleção de Variáveis

Considerar técnicas como regressão lasso, ridge ou elastic net, que fazem seleção de variáveis automaticamente, pode ser útil se você suspeitar que nem todas as 20 variáveis são relevantes.

In [4]:
df_model.columns

Index(['BalanceSheet_global', 'BTC_USD', 'Mining_Difficulty',
       'USD_OnChain_Volume', 'GDP_usa', 'HashRate', 'M2_Liquidity',
       'Dollar_Index', 'QQQ', 'GLD', 'TLT', '10YR_USA', 'VIX', 'OIL_USD',
       'Miners_Revenue_USD', 'Nr_Tx_Block', 'Nr_Transactions', 'Nr_Addresses',
       'BTC_PageViews', 'Crypto_PageViews', 'BTC_Supply',
       'L1.BalanceSheet_global', 'L2.BalanceSheet_global', 'L1.BTC_USD',
       'L2.BTC_USD', 'L1.Mining_Difficulty', 'L2.Mining_Difficulty',
       'L1.USD_OnChain_Volume', 'L2.USD_OnChain_Volume', 'L1.GDP_usa',
       'L2.GDP_usa', 'L1.HashRate', 'L2.HashRate', 'L1.M2_Liquidity',
       'L2.M2_Liquidity', 'L1.Dollar_Index', 'L2.Dollar_Index', 'L1.QQQ',
       'L2.QQQ', 'L1.GLD', 'L2.GLD', 'L1.TLT', 'L2.TLT', 'L1.10YR_USA',
       'L2.10YR_USA', 'L1.VIX', 'L2.VIX', 'L1.OIL_USD', 'L2.OIL_USD',
       'L1.Miners_Revenue_USD', 'L2.Miners_Revenue_USD', 'L1.Nr_Tx_Block',
       'L2.Nr_Tx_Block', 'L1.Nr_Transactions', 'L2.Nr_Transactions',
       'L1.Nr_Ad

<div class="alert alert-block alert-info"><b>Hipoteses a testar</b>

H1: Ceteris paribus, Bitcoin é um ativo de especulação.

H2: Ceteris paribus, a performance de Bitcoin é influenciada por fatores macroeconómicos.

H3: Ceteris paribus, quanto mais liquidez há no mercado, maior é a apreciação de preço de Bitcoin.

H4: Ceteris paribus, os fatores endógenos de Bitcoin são os maiores explicadores da performance de Bitcoin.
</div>

## Regressao Linear Multipla

In [9]:
# Yt = b0 + b1X1t + b2X2t + εt

#   L1.
#   L2.


# Modelos endogenos Bitcoin
#   'Mining_Difficulty', 'USD_OnChain_Volume', 'HashRate', 'Miners_Revenue_USD', 
#   'Nr_Tx_Block', 'Nr_Transactions', 'Nr_Addresses', 'BTC_Supply'
model1a = ['Mining_Difficulty', 'USD_OnChain_Volume', 'BTC_Supply', 'Nr_Transactions']
model1b = ['Nr_Transactions', 'L1.USD_OnChain_Volume', 'L1.BTC_Supply', 'L1.Mining_Difficulty']
model1c = ['Nr_Tx_Block', 'L2.USD_OnChain_Volume']


# Modelos Macro Economicos
#   'GDP_usa', 'Dollar_Index', 'QQQ', 'GLD', 'TLT', '10YR_USA', 'VIX'
#   'BalanceSheet_global', 'M2_Liquidity', 'OIL_USD'
model2a = [ 'QQQ', 'OIL_USD', 'GLD', 'BalanceSheet_global']
model2b = [ 'GLD', 'VIX', 'M2_Liquidity', 'TLT']
model2c = [ 'TLT', 'GDP_usa', 'BalanceSheet_global', 'L1.GLD']
model2d = [ '10YR_USA', 'L1.VIX', 'L1.OIL_USD', 'Dollar_Index' ]


# Modelos Gerais
#   'BTC_PageViews', 'Crypto_PageViews', 'L1.BTC_USD'
model3a = [  'VIX', '10YR_USA', 'HashRate', 'BTC_Supply'] #'BTC_PageViews',
model3b = [  'Nr_Transactions', 'L1.QQQ', 'M2_Liquidity'] # 'L1.BTC_PageViews', 'Crypto_PageViews',
model3c = [  'L2.GLD', 'BalanceSheet_global', 'OIL_USD', 'Miners_Revenue_USD', 'USD_OnChain_Volume'] #'L2.BTC_PageViews',


X = df_model[model1b]
y = df_model["BTC_USD"]

X = sm.add_constant(X) # Add a constant term to the predictors

model = sm.OLS(y, X)
results = model.fit() # Fit the OLS model

print(results.summary()) # Print the summary of the regression results

                            OLS Regression Results                            
Dep. Variable:                BTC_USD   R-squared:                       0.716
Model:                            OLS   Adj. R-squared:                  0.710
Method:                 Least Squares   F-statistic:                     126.6
Date:                Tue, 27 Aug 2024   Prob (F-statistic):           8.69e-54
Time:                        19:12:43   Log-Likelihood:                 139.27
No. Observations:                 206   AIC:                            -268.5
Df Residuals:                     201   BIC:                            -251.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  -175.24

### Auto run Regressions

In [41]:
models_list = [model1a, model1b, model1c, model2a, model2b, 
               model2c, model2d, model3a, model3b, model3c]

models_columns = ['model_1a', 'model_1b', 'model_1c', 'model_2a', 'model_2b', 
               'model_2c', 'model_2d', 'model_3a', 'model_3b', 'model_3c']

def run_models_stats(df, models_list, y):
        count = 0
        for model_columns in models_list:
            X = df[model_columns]

            X = sm.add_constant(X)  # Add a constant term to the predictors

            model = sm.OLS(y, X)
            results = model.fit()  # Fit the OLS model
            
            print("\n####################################################################################################### \n\n")
            print("\nModel:", models_columns[count]) # columns name
            #print("Variable:", results.params.index) # set as index
            print(f"\nCoeff: {results.params}") # coefficients
            print("\nP-Value:", results.pvalues) # p-values
            print("\nR-Squared:", results.rsquared_adj)
            print("No. Obs.:", results.nobs)
            print("Cond. no.:", results.condition_number)
            count += 1
            print("\n####################################################################################################### \n\n")


def run_models(df, models_list, y):
    f = open(f"MLR_Results_Ciclo{ciclo}.txt", "w")
    count = 0
    for model_columns in models_list:
        X = df[model_columns]

        X = sm.add_constant(X)  # Add a constant term to the predictors

        model = sm.OLS(y, X)
        results = model.fit()  # Fit the OLS model
        
        f.write("\n####################################################################################################### \n\n")
        f.write(models_columns[count])
        f.write(" \n")
        f.write(f"{results.summary()}")
        count += 1
        f.write("\n####################################################################################################### \n\n")
    
    f.close()

In [43]:
y = opencsv["BTC_USD"]
run_models_stats(df_model, models_list, y)


####################################################################################################### 



Model: model_1a

Coeff: const                -164.370812
Mining_Difficulty      -0.811650
USD_OnChain_Volume      0.761784
BTC_Supply             23.680239
Nr_Transactions         0.016231
dtype: float64

P-Value: const                 1.190030e-10
Mining_Difficulty     4.592658e-07
USD_OnChain_Volume    3.977090e-53
BTC_Supply            2.615170e-10
Nr_Transactions       8.845836e-01
dtype: float64

R-Squared: 0.7325439114332171
No. Obs.: 206.0
Cond. no.: 55485.67309048347

####################################################################################################### 



####################################################################################################### 



Model: model_1b

Coeff: const                   -175.244090
Nr_Transactions            0.160223
L1.USD_OnChain_Volume      0.756563
L1.BTC_Supply             25.252734
L1.Mining_Difficulty      

In [ ]:
#run_models_stats(df_model, models_list, y)

### Exemplo Data Frame

In [ ]:
data_store = pd.DataFrame()
data_store.index = df_model.columns


models_columns = ['model1a', 'model1b', 'model1c', 'model1d', 'model2a', 'model2b', 
               'model2c', 'model2d', 'model2e', 'model3a', 'model3b', 'model3c']

# Add new columns with a default value (e.g., None or 0)
for model in models_columns:
    data_store[model] = None  # or some other default value

data_store

In [ ]:
# list of all params 
dir(results)

https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html

# Testes

### Homocesdasticity

In [ ]:
import statsmodels.formula.api as smf

# https://www.geeksforgeeks.org/how-to-perform-a-breusch-pagan-test-in-python/

### Normality of Residuals

Assumption: The residuals should be approximately normally distributed.

Test:
Q-Q Plot: Compare the quantiles of residuals to a normal distribution. If residuals are normal, the points will lie approximately on the 45-degree line.

Shapiro-Wilk Test or Kolmogorov-Smirnov Test: Statistical tests for normality of residuals.

Transformation: If residuals are not normally distributed, consider transforming the dependent variable or using robust regression methods.

### Independence of Errors


Assumption: The residuals (errors) should be independent of each other.

Test:

Durbin-Watson Test: Detects the presence of autocorrelation in residuals, especially in time series data.

Plot Residuals vs. Time: For time series data, plot residuals against time to detect patterns or autocorrelation.

Transformation:
Add Lagged Variables: For time series, include lagged values of the dependent variable as predictors.

### No Omitted Variable Bias

Assumption: All relevant independent variables are included in the model.

Test:

Ramsey RESET Test: Checks for omitted variables by testing for non-linearity in the model.

Remedies:

Include Relevant Variables: Ensure all theoretically important variables are included in the model.
Use Model Selection Criteria: Such as AIC or BIC to guide variable selection.

## Teste de Multicolinearidade


Como você tem muitas variáveis, é importante verificar a multicolinearidade, especialmente porque as variáveis acumuladas podem estar altamente correlacionadas entre si. Testes como o VIF (Variance Inflation Factor) podem ajudar a identificar problemas.

Multicollinearity describes the state where the independent variables used in a study exhibit a strong relationship with each other. This can pose a problem in many cases as you would normally want your independent variables to be… independent.

In [11]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

#find design matrix for linear regression model using 'rating' as response variable 
y, X = dmatrices('BTC_USD ~ Nr_Transactions + L1.USD_OnChain_Volume + L1.BTC_Supply + L1.Mining_Difficulty', data=df_model, return_type='dataframe')

#calculate VIF for each explanatory variable
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns

#view VIF for each explanatory variable 
vif


#       Interpreting the VIF Values:
#       VIF < 5: Generally considered acceptable, indicating low to moderate multicollinearity.
#       VIF between 5 and 10: Indicates moderate to high multicollinearity, which may be problematic, but not necessarily requiring immediate removal.
#       VIF > 10: Strong multicollinearity, typically suggesting that the variable should be considered for removal or that further investigation is needed.

PatsyError: Error evaluating factor: NameError: name 'L1' is not defined
    BTC_USD ~ Nr_Transactions + L1.USD_OnChain_Volume + L1.BTC_Supply + L1.Mining_Difficulty
                                                        ^^^^^^^^^^^^^